# Chapter 15. K-Nearest Neighbors

## 15.0 Introduction
* KNN : one of the simplest yet most commonly used classifiers in supervised ML.

## 15.1 Finding an Observation's Nearest Neighbors

* Find an obs's k nearest observations(Neighbors)
* scikit-learn's NearestNeighbors

In [4]:
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data

standardizer = StandardScaler()

# Standardize features
features_standardized = standardizer.fit_transform(features)

# Two nearest neighbors
nearest_neighbors = NearestNeighbors(n_neighbors=2).fit(features_standardized)

# Create an observation
new_observation = [ 1, 1, 1, 1]

# Find distances and indices of the observation's NN.
distances, indices = nearest_neighbors.kneighbors([new_observation])

# View the NN
features_standardized[indices]

array([[[1.03800476, 0.55861082, 1.10378283, 1.18556721],
        [0.79566902, 0.32841405, 0.76275827, 1.05393502]]])

$$d_{euclidean} = \sqrt{\sum_{i=1}^n (x_i - y_i)^2}$$

* NearestNeighbors : Minkowski 거리를 사용한다.
    * 조정하기 위해서는 metric 파라미터를 활용한다.

In [8]:
# Find each obs's three nearest neighbors
# 유클리드 거리 기반으로 찾아 본다.

nearestneighbors_euclidean = NearestNeighbors(
    n_neighbors=3, metric="euclidean").fit(features_standardized)

# List of lists indicating each obs's 3 NNs
nearest_neighbors_with_self = nearestneighbors_euclidean.kneighbors_graph(
    features_standardized).toarray()

# 관측값을 1로 마킹된 것을 제거한다.
for i, x in enumerate(nearest_neighbors_with_self):
    x[i] = 0

# Vuew first obs's two nearest neighbors
nearest_neighbors_with_self[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## 15.2 Creating a K-Nearest Neighbor Classifier

* 데이터셋이 매우 크지 않으면, KNeighborsClassifier 활용한다.

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

standardizer = StandardScaler()

# 피처를 표준화 한다.
X_std = standardizer.fit_transform(X)

# KNN classifiers with 5 neighbors로 훈련한다.
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1).fit(X_std, y)

# 2 obs 제작
new_observations = [[0.75, 0.75, 0.75, 0.75], [1,1,1,1]]

# Predict the class of two obs
knn.predict(new_observations)

array([1, 2])

* KNeighborsClassifier의 파라미터
    * metric : 거리 기준을 결정한다.
    * n_jobs : 컴퓨터 코어 수를 결정한다.
    * algorithm : NN을 계산하는 방법을 결정한다.
        * 물론 알고리즘은 이 분류기가 결정한다.(디폴트)
    * 각 이웃은 하나의 보트를 수행하고, 다만 weights로 거리가 설정된다면, 더 가까운 관측값의 보트가 더 가중치를 갖는다.
    * 더 비슷한 이웃이 더 많이 클래스를 말해주기 때문이다.

## 15.3 Identifying the Best Neighborhood Size

* Best value for k in a KNN Classifier.
* GridSearchCV

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

iris = datasets.load_iris()
features = iris.data
target = iris.target

# Create standardizer
standardizer = StandardScaler()

# Create a KNN classifier
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

# Create a pipeline
pipe = Pipeline([("standardizer", standardizer), ("knn", knn)])

# Create space of candidate values : Dict 타입으로, 표기명에 주의한다.
search_space = [{"knn__n_neighbors": [1,2,3,4,5,6,7,8,9,10]}]

# Grid search
classifier = GridSearchCV(
    pipe, search_space, cv=5, verbose=0).fit(features_standardized, target)

* k의 크기는 KNN classifiers에 큰 의미를 가진다.
* ML에서 우리는 bias, variance 사이에 균형을 찾고자 하고, k의 값이 명확하지 않다.
* 이 때, Grid search 를 통해서 여러 k 값으로 교차 검증을 해볼 수 있다.(5-fold cv)

In [15]:
# Best neighborhood size (k)
classifier.best_estimator_.get_params()["knn__n_neighbors"]

6

## 15.4 Creating a Radius-Based Nearest Neighbor Classifier

* unknown class를 갖는 관측값에서, 일정한 거리 기반으로 모든 관측값의 클래스를 예측하고 싶다.
* 거리가 나오면, Radius를 생각한다.

In [2]:
# RadiusNeighborsClassifier

from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

iris = datasets.load_iris()
features = iris.data
target = iris.target

# Create standardizer
standardizer = StandardScaler()

# Standardize features
features_standardized = standardizer.fit_transform(features)

# 이제, 방사 이웃 분류기를 훈련 시킨다. 줄이면 rnn이라 한다.
rnn = RadiusNeighborsClassifier(
    radius=.5, n_jobs=-1).fit(features_standardized, target)

# 2 obs
new_observations = [[1, 1, 1, 1]]

# Predict the class of 2 obs
rnn.predict(new_observations)

array([2])

* KNN classification
    * k개 이웃들의 클래스에서 예측된다.
* radius-based (RNN) classifier : 주어진 radius r 내에 모든 관측값의 클래스를 예측한다.
    * 사이킷 런에서 제공하고 있고, 2가지 파라미터가 차이가 있다.(KNN 비교)
        * radius를 지정해야 한다.
        * outlier_label : no obs인 경우 관측값에 주어지는 라벨이다. 이상치(outlier) 탐색에 유용하다.